In [13]:
from docx import Document

In [9]:
import nltk
from nltk.corpus import brown
from nltk.corpus import stopwords
from pymystem3 import Mystem
import numpy as np
import pandas as pd

In [10]:
from os import listdir

## Cоздаём удобный файл для ассесора

In [5]:
#read docx files from folder and save them to one big txt file
def set_cvs_charter_statutes(folder):
    file_list = listdir(folder)
    with open('assessor_charters.txt', 'w', encoding='utf-8') as f:            
        for charter_name in file_list:
            charter = Document(folder+'/'+charter_name)
            for paragraph in charter.paragraphs:                                
                if len(paragraph.text): f.writelines(paragraph.text+'\n')

In [36]:
#find all listings of document statutes and collapse them in one line
def preprocess_txt_charter(file_name):    
    new_text = []    
    flag = False #marker if need to collaps statute
    with open(file_name, encoding='utf-8') as f:                
        for line in f.readlines():
            if flag: 
                if not line.endswith('.\n'):
                    statute.append(line.strip('\n'))
                else:
                    flag = False
                    new_text.append(' '.join(statute))
                    statute = []
            else:
                if line.endswith(':\n'):
                    flag = True
                    statute = [line.strip('\n')]
                else:
                    new_text.append(line)            
                    
    with open('preprocessed_assessor_charters.txt', 'w', encoding='utf-8') as f:
        f.write("".join(new_text))

In [ ]:
charters = [Document('charter.docx'), Document('charter_doc.docx')]
set_csv_charter(charters)

In [16]:
set_cvs_statutes('Charters')

In [37]:
preprocess_txt_charter('assessor_charters.txt')

## Определяем datasets. Проводим лемматизацию c помощью MyStem

In [17]:
df = pd.read_csv('ready_assessor_charter.csv', sep=';')

In [18]:
texts = df.iloc[:, 1].values
target = df.iloc[:, 0].values

In [19]:
df.text.isna().any()

False

In [20]:
import pymorphy2
import re # Регулярные выражения.
from collections import Counter


import re # Регулярные выражения.
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import pymorphy2 # Морфологический анализатор.
import datetime # Новости будем перебирать по дате.
from collections import Counter # Не считать же частоты самим.
import math # Корень квадратный.

In [21]:
morph=pymorphy2.MorphAnalyzer()

In [22]:
morph.normal_forms('собраний')

['собрание']

In [82]:
posConv={'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB', 'PRTS':'_PRTS'}

def getArticleDictionary(text, needPos=None):
    words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
    reswords=[]

    for w in words:
        wordform=morph.parse(w)[0]
        if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB', 'PRTS']:
            if needPos!=None:
                reswords.append(wordform.normal_form+posConv[wordform.tag.POS])
            else:
                reswords.append(wordform.normal_form)            
    return reswords

In [83]:
%%time
nom_texts = []
for string in texts:                   
    dictt = " ".join(getArticleDictionary(string, needPos=True))    
    print (string, '\n', dictt, '\n')
    nom_texts.append(dictt)

Общие положения 
 общий_ADJ положение_NOUN 

Общество с ограниченной ответственностью «Инновационные системы управления», именуемое в дальнейшем «Общество», создано и действует в соответствии с Гражданским кодексом Российской Федерации, Федеральным законом от 08.02.1998 N 14-ФЗ "Об обществах с ограниченной ответственностью" и настоящим уставом, именуемым в дальнейшем - «Устав». 
 общество_NOUN ограниченный_ADJ ответственность_NOUN инновационный_ADJ система_NOUN управление_NOUN дальнейший_ADJ общество_NOUN создать_PRTS действовать_VERB соответствие_NOUN гражданский_ADJ кодекс_NOUN российский_ADJ федерация_NOUN федеральный_ADJ закон_NOUN общество_NOUN ограниченный_ADJ ответственность_NOUN настоящее_NOUN устав_NOUN дальнейший_ADJ 

Наименование Общества. 
 наименование_NOUN общество_NOUN 

Полное фирменное наименование: 
 полный_ADJ фирменный_ADJ наименование_NOUN 

Общество с ограниченной ответственностью «Инновационные системы управления» 
 общество_NOUN ограниченный_ADJ ответственность

Увеличение уставного капитала Общества может осуществляться за счет имущества Общества, за счет дополнительных вкладов участников Общества, а также за счёт вкладов третьих лиц, принимаемых в Общество. 
 увеличение_NOUN уставный_ADJ капитал_NOUN общество_NOUN мочь_VERB счёт_NOUN имущество_NOUN общество_NOUN счёт_NOUN дополнительный_ADJ вклад_NOUN участник_NOUN общество_NOUN счёт_NOUN вклад_NOUN третье_NOUN лицо_NOUN общество_NOUN 

Увеличение уставного капитала Общества за счет имущества Общества: 
 увеличение_NOUN уставный_ADJ капитал_NOUN общество_NOUN счёт_NOUN имущество_NOUN общество_NOUN 

Увеличение уставного капитала Общества за счет его имущества осуществляется по решению общего собрания участников Общества, принятому единогласно. При этом, решение об увеличении уставного капитала Общества за счет имущества Общества может быть принято только на основании данных бухгалтерской отчетности Общества за год, предшествующий году, в течение которого принято такое решение. 
 увеличение_N

In [43]:
nom_texts

['общий_ADJ положение_NOUN',
 'общество_NOUN ограниченный_ADJ ответственность_NOUN инновационный_ADJ система_NOUN управление_NOUN дальнейший_ADJ общество_NOUN действовать_VERB соответствие_NOUN гражданский_ADJ кодекс_NOUN российский_ADJ федерация_NOUN федеральный_ADJ закон_NOUN общество_NOUN ограниченный_ADJ ответственность_NOUN настоящее_NOUN устав_NOUN дальнейший_ADJ',
 'наименование_NOUN общество_NOUN',
 'полный_ADJ фирменный_ADJ наименование_NOUN',
 'общество_NOUN ограниченный_ADJ ответственность_NOUN инновационный_ADJ система_NOUN управление_NOUN',
 'фирменный_ADJ наименование_NOUN',
 'ооо_NOUN инновационный_ADJ система_NOUN управление_NOUN',
 'место_NOUN нахождение_NOUN общество_NOUN определяться_VERB место_NOUN государственный_ADJ регистрация_NOUN территория_NOUN российский_ADJ федерация_NOUN место_NOUN нахождение_NOUN действующий_ADJ исполнительный_ADJ орган_NOUN',
 'рф_NOUN город_NOUN москва_NOUN город_NOUN зеленоград_NOUN',
 'правовой_ADJ статус_NOUN общество_NOUN',
 'обществ

## создание векторов с уменьшением размера частых слов с помощью sklearn

In [44]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import pandas as pd

In [52]:
model = Pipeline([
    ('vect',  CountVectorizer(ngram_range=(1, 3), max_df=0.8, min_df=3)),
    ('clf', LogisticRegression())
])

In [53]:
model.fit(nom_texts, target)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=3,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [54]:
words = model.steps[0][1].get_feature_names()

In [55]:
len(words)

917

In [56]:
coefs = model.steps[1][1].coef_[0]

In [57]:
model_weigths = pd.Series(index=words, data=coefs)

In [58]:
model_weigths.sort_values()

доля_noun                                      -0.647797
внеочередной_adj                               -0.637317
внеочередной_adj общий_adj                     -0.637317
внеочередной_adj общий_adj собрание_noun       -0.637317
мочь_verb                                      -0.522615
право_noun                                     -0.500481
решение_noun общий_adj собрание_noun           -0.474540
решение_noun общий_adj                         -0.474540
деятельность_noun                              -0.402934
уставный_adj                                   -0.390802
часть_noun                                     -0.365934
требование_noun                                -0.365077
управление_noun                                -0.361712
месяц_noun                                     -0.343657
законодательство_noun                          -0.341316
счёт_noun                                      -0.330491
определение_noun                               -0.307802
который_adj                    

In [74]:
text_string = 'Орган или лица, созывающие общее собрание участников Общества, обязаны не позднее чем за тридцать дней до его проведения уведомить об этом каждого участника Общества заказным письмом по адресу, указанному в списке участников Общества.'

In [75]:
nom_text_string = " ".join(getArticleDictionary(text_string, needPos=True))

In [76]:
nom_text_string

'орган_NOUN лицо_NOUN общий_ADJ собрание_NOUN участник_NOUN общество_NOUN поздний_ADJ день_NOUN проведение_NOUN каждый_ADJ участник_NOUN общество_NOUN заказной_ADJ письмо_NOUN адрес_NOUN указанный_ADJ список_NOUN участник_NOUN общество_NOUN'

In [77]:
vect = CountVectorizer(ngram_range=(1, 3), max_df=0.8, min_df=3)

In [78]:
model.predict_proba([nom_text_string])

array([[0.15424127, 0.84575873]])

## ARTM

In [ ]:
with open('charter.txt', 'w', encoding='utf-8') as f:
    for string in new_text:
        #print(string)
        f.write("|text" +" "+string+"\n")

In [ ]:
import artm
# создание частотной матрицы из batch
batch_vectorizer = artm.BatchVectorizer(data_path='charter.txt',# путь к "мешку слов"
                                        data_format='vowpal_wabbit',# формат данных
                                       target_folder='charter', #папка с частотной матрицей из batch
                                        batch_size=50)# количество документов в одном batch

In [ ]:
batch_vectorizer = artm.BatchVectorizer(data_path='charter',data_format='batches')
dictionary = artm.Dictionary(data_path='charter')# загрузка данных в словарь

model = artm.ARTM(num_topics=20,
                  num_document_passes=10,#10 проходов по документу
                  dictionary=dictionary,
                  scores=[artm.TopTokensScore(name='top_tokens_score')])

In [ ]:
model.initialize(dictionary=dictionary)

In [ ]:
model.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=10)#10 проходов по коллекции


In [ ]:
top_tokens = model.score_tracker['top_tokens_score']

In [ ]:
print(top_tokens.last_tokens)

In [ ]:
for topic_name in model.topic_names:
    print(topic_name)
    for (token, weight) in zip(top_tokens.last_tokens[topic_name], top_tokens.last_weights[topic_name]):
         print(token, '-', round(weight,3))

In [ ]:
top_tokens.tokens